In [ ]:
from pyquil.simulation.matrices import I, X, Y, Z
import numpy as np
def sequence_list_length_by_1():

def dephaseing_kraus(p):
    A=[np.kron(i_deph,j_deph) for i_deph in [I,Z] for j_deph in [I,Z]]
    for i_deph in [0,3]:
        A[i_deph] = A[i_deph]*(1-p)
    for i_deph in [1,2]:
        A[i_deph] = A[i_deph]*np.sqrt(p*(1-p))
    return A

def depolarizing_kraus(p):
    A=[np.kron(i_depo,j_depo) for i_depo in [I,X,Y,Z] for j_depo in [I,X,Y,Z]]
    A[0] = A[0]*np.sqrt(1-p)
    for i_depo in range(len(A[1:])):
        A[i_depo]=A[i_depo]*np.sqrt(p/15)
    return A
if __name__ == '__main__':
####### input
    rb_types=int(input('-1 rb, 0 decomp_rb, 1 prb\n'))
    max_seq= int(input('max seq \n'))
####### rb gate number i    
    for i in tqdm(sequence_length,ascii=True,desc="sequence_length",dynamic_ncols=True):
####### generate rb sequence        
        pool = Pool()
        seq = pool.map(sequence_list_length_by_1,[i]*types)
        pool.close()
        pool.join()
####### rb sequence into probablility
        types_probability=[]
        m_average=[]
        pool = mp.Pool()
        repeat_probability = pool.starmap(sequence_to_probability,[(i,None,d,e) for d in range(1) for e in seq ])
        pool.close()
        pool.join()

In [34]:
from pyquil.simulation.matrices import I, X, Y, Z
import numpy as np
from forest.benchmarking.operator_tools import kraus2pauli_liouville
from print_large import _print_big_matrix
def dephaseing_kraus(p):
    A=[np.kron(i_deph,j_deph) for i_deph in [I,Z] for j_deph in [I,Z]]
    for i_deph in [0,3]:
        A[i_deph] = A[i_deph]*(1-p)
    for i_deph in [1,2]:
        A[i_deph] = A[i_deph]*np.sqrt(p*(1-p))
    return A
def depolarizing_kraus(p):
    A=[np.kron(i_depo,j_depo) for i_depo in [I,X,Y,Z] for j_depo in [I,X,Y,Z]]
    A[0] = A[0]*np.sqrt(1-p)
    for i_depo in range(1,len(A)):
        A[i_depo]=A[i_depo]*np.sqrt(p/15)
    return A
if __name__ == '__main__':

    a=kraus2pauli_liouville(dephaseing_kraus(0.9))
    _print_big_matrix(a)



0.2  .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .  0.2  .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .  -0.2  .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .  -0.2  .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .  -0.2  .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .  -0.2  .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .  0.2  .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   

In [52]:
from pyquil.simulation.matrices import I, X, Y, Z
import numpy as np
from forest.benchmarking.operator_tools import kraus2pauli_liouville
from print_large import _print_big_matrix

def densityop_01_2_pauli(density_op):
    a_00=np.trace(density_op)
    a_10=np.trace(X@density_op)
    a_20=np.trace(Y@density_op)
    a_30=np.trace(Z@density_op)

    return np.array([a_00, a_10, a_20, a_30]).reshape(4,1)
if __name__ == '__main__':
    density_op=np.asarray([[1,0],[0,0]])
    print(densityop_01basis_to_paulibasis(density_op))
    _print_big_matrix(np.kron(densityop_01basis_to_paulibasis(density_op),densityop_01basis_to_paulibasis(density_op)))

[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [1.+0.j]]
1.0 
 .  
 .  
1.0 
 .  
 .  
 .  
 .  
 .  
 .  
 .  
 .  
1.0 
 .  
 .  
1.0 
